In [70]:
# Define the model using functional API:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Input,Reshape,Flatten
import keras
import numpy as np

layers = [Input(shape=(256,256,1)),
      Reshape((64,64,-1)),
      Conv2D(filters=32,kernel_size=5),
      MaxPool2D(2),
      Conv2D(filters=1,kernel_size=1),
      Flatten(),
      Dense(16,activation='sigmoid')]
model = Sequential(layers)
model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_28 (Reshape)        (None, 64, 64, 16)        0         
                                                                 
 conv2d_60 (Conv2D)          (None, 60, 60, 32)        12832     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 30, 30, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_61 (Conv2D)          (None, 30, 30, 1)         33        
                                                                 
 flatten_4 (Flatten)         (None, 900)               0         
                                                                 
 dense_4 (Dense)             (None, 16)                14416     
                                                     

In [71]:
# Define the model using functional API:
from keras.models import Model
from keras.layers import Dense,Conv2D,MaxPool2D,Input,Reshape,Flatten,concatenate
import keras
import numpy as np

def my_model():
  # define layers
    input_tensor = Input(shape=(256,256,1))
    reshape_layer = Reshape((64,64,-1))
    conv2d_1 = Conv2D(filters=8,kernel_size=5)
    conv2d_2 = Conv2D(filters=8,kernel_size=17)
    max_pool_layer = MaxPool2D(4)

    input_tensor_reshape = reshape_layer(input_tensor)
    conv1_out = conv2d_1(input_tensor_reshape)
    conv2_out = conv2d_2(input_tensor)
    max_pool_out = max_pool_layer(conv2_out)
    cat_out = concatenate([conv1_out,max_pool_out],axis=-1)
    model = Model(inputs=input_tensor, outputs=cat_out)
    return model

my_model = my_model()
my_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_46 (InputLayer)          [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 reshape_29 (Reshape)           (None, 64, 64, 16)   0           ['input_46[0][0]']               
                                                                                                  
 conv2d_63 (Conv2D)             (None, 240, 240, 8)  2320        ['input_46[0][0]']               
                                                                                                  
 conv2d_62 (Conv2D)             (None, 60, 60, 8)    3208        ['reshape_29[0][0]']       

In [72]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.losses import SparseCategoricalCrossentropy

df_orig_train = pd.read_csv('./mnist_train.csv',header=None)
df_orig_test = pd.read_csv('./mnist_test.csv',header=None)
df_train_values = df_orig_train.values
df_test_values = df_orig_test.values
train_feat_ori,train_label_ori = df_train_values[:,1:]/255.0,df_train_values[:,0]
test_feat,test_label = df_test_values[:,1:]/255.0,df_test_values[:,0]
train_feat,val_feat = train_feat_ori[6000:],train_feat_ori[:6000]
train_label,val_label = train_label_ori[6000:],train_label_ori[:6000]

model = Sequential()
model.add(Input(shape=(784,)))
model.add(Dense(512, activation='relu')) 
model.add(Dense(512, activation='relu')) 
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size = 128
epochs = 8
model.fit(train_feat, train_label, batch_size=batch_size, epochs=epochs, validation_data = (val_feat,val_label))

Epoch 1/8
422/422 [==============================] - 2s 4ms/step - loss: 0.2306 - accuracy: 0.9314 - val_loss: 0.1007 - val_accuracy: 0.9700
Epoch 2/8
422/422 [==============================] - 1s 3ms/step - loss: 0.0846 - accuracy: 0.9734 - val_loss: 0.0789 - val_accuracy: 0.9745
Epoch 3/8
422/422 [==============================] - 1s 3ms/step - loss: 0.0539 - accuracy: 0.9827 - val_loss: 0.0695 - val_accuracy: 0.9785
Epoch 4/8
422/422 [==============================] - 1s 3ms/step - loss: 0.0376 - accuracy: 0.9882 - val_loss: 0.0821 - val_accuracy: 0.9762
Epoch 5/8
422/422 [==============================] - 1s 3ms/step - loss: 0.0284 - accuracy: 0.9911 - val_loss: 0.0775 - val_accuracy: 0.9750
Epoch 6/8
422/422 [==============================] - 1s 3ms/step - loss: 0.0213 - accuracy: 0.9932 - val_loss: 0.0702 - val_accuracy: 0.9805
Epoch 7/8
422/422 [==============================] - 1s 3ms/step - loss: 0.0174 - accuracy: 0.9941 - val_loss: 0.0749 - val_accuracy: 0.9802
Epoch 8/8
422

In [75]:
df_train_values[:,0]

array([5, 0, 4, ..., 5, 6, 8], dtype=int64)

In [95]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.losses import SparseCategoricalCrossentropy

import numpy as np
import linecache
import random
class DataGenerator(keras.utils.all_utils.Sequence):
#   'Generates data for Keras'
    def __init__(self, csv_path, indexes, bs):
    # initilizes some variables 
        self.csv_path = csv_path
        self.norm_facor = 255.0
        self.indexes = indexes
        self.bs = bs
        random.shuffle(self.indexes)
    def __len__(self):
    # return the total number of samples in the dataset
        return (len(self.indexes))//self.bs-1
    def __getitem__(self,index):
    # get one sample according to the index
        feat_all = []
        label_all = []
        for this_index in range(index*self.bs,(index+1)*self.bs):
            line_index = self.indexes[index]
            line_str = linecache.getline(self.csv_path,line_index)
#             line_str.replace('[','').replace(']', '')
#             print(line_str)
            line_val = [int(i) for i in line_str.split(',') if len(i)]
#             print(line_val)
            label = line_val[0]
            feat = np.array(line_val[1:])/self.norm_facor
            feat_all.append(feat)
            label_all.append(label)

        return np.array(feat_all),np.array(label_all)
    def shuffle(self):
        random.shuffle(self.indexes)


indexes = [i for i in range(60000)]
train_index = indexes[6000:]
val_index = indexes[:6000]
batch_size = 128
train_set = DataGenerator('./mnist_train.csv',train_index,batch_size)
val_set = DataGenerator('./mnist_train.csv',val_index,batch_size)

model = Sequential()
model.add(Input(shape=(784,)))
model.add(Dense(512, activation='relu')) 
model.add(Dense(512, activation='relu')) 
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy',metrics=['accuracy'])

epochs = 15
for i_epoch in range(epochs):
    print('[{}/{}]'.format(i_epoch,epochs))
    model.fit(train_set, steps_per_epoch = len(train_set), epochs=1, validation_data = val_set,validation_steps=len(val_set))
    train_set.shuffle()

[0/15]
420/420 [==============================] - 9s 20ms/step - loss: 1.3219 - accuracy: 0.6071 - val_loss: 0.8920 - val_accuracy: 0.6444
[1/15]
420/420 [==============================] - 8s 20ms/step - loss: 0.7430 - accuracy: 0.7286 - val_loss: 0.5863 - val_accuracy: 0.7778
[2/15]
420/420 [==============================] - 8s 20ms/step - loss: 0.5262 - accuracy: 0.8381 - val_loss: 0.5150 - val_accuracy: 0.8000
[3/15]
420/420 [==============================] - 8s 20ms/step - loss: 0.5143 - accuracy: 0.8548 - val_loss: 0.6014 - val_accuracy: 0.8222
[4/15]
420/420 [==============================] - 9s 20ms/step - loss: 0.5481 - accuracy: 0.8452 - val_loss: 0.4123 - val_accuracy: 0.9333
[5/15]
420/420 [==============================] - 8s 20ms/step - loss: 0.4938 - accuracy: 0.8667 - val_loss: 0.2825 - val_accuracy: 0.9333
[6/15]
420/420 [==============================] - 9s 20ms/step - loss: 0.3668 - accuracy: 0.8857 - val_loss: 0.5664 - val_accuracy: 0.8667
[7/15]
420/420 [===========

In [ ]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Input

import numpy as np
import linecache
import random
import tensorflow as tf

class DataGenerator(keras.utils.all_utils.Sequence):
#   'Generates data for Keras'
    def __init__(self, csv_path, indexes, bs):
    # initilizes some variables 
        self.csv_path = csv_path
        self.norm_facor = 255.0
        self.indexes = indexes
        self.bs = bs
        random.shuffle(self.indexes)
    def __len__(self):
    # return the total number of samples in the dataset
        return (len(self.indexes))//self.bs-1
    def __getitem__(self,index):
    # get one sample according to the index
        feat_all = []
        label_all = []
        for this_index in range(index*self.bs,(index+1)*self.bs):
            line_index = self.indexes[index]
            line_str = linecache.getline(self.csv_path,line_index)
#             line_str.replace('[','').replace(']', '')
#             print(line_str)
            line_val = [int(i) for i in line_str.split(',') if len(i)]
#             print(line_val)
            label = line_val[0]
            feat = np.array(line_val[1:])/self.norm_facor
            feat_all.append(feat)
            label_all.append(label)

        return np.array(feat_all),np.array(label_all)
    def shuffle(self):
        random.shuffle(self.indexes)

indexes = [i for i in range(60000)]
train_index = indexes[6000:]
val_index = indexes[:6000]
batch_size = 128
train_set = DataGenerator('./mnist_train.csv',train_index,batch_size)
val_set = DataGenerator('./mnist_train.csv',val_index,batch_size)

model = Sequential()
model.add(Input(shape=(784,)))
model.add(Dense(512, activation='relu')) 
model.add(Dense(512, activation='relu')) 
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam',loss= 'sparse_categorical_crossentropy',metrics=['accuracy'])

loss_fn = keras.losses.SparseCategoricalCrossentropy()
# optimizer = keras.optimizer_v1.Adam()
optimizer = tf.keras.optimizers.Adam()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
epochs = 50
best_metric = 100
for epoch in range(epochs):
    train_set.shuffle()
    for step, (x_batch_train, y_batch_train) in enumerate(train_set):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    for x_batch_val, y_batch_val in val_set:
        val_logits = model(x_batch_val, training=False)
        val_acc_metric.update_state(y_batch_val, val_logits)
        val_acc = val_acc_metric.result()
        print("[{}/{}],val_acc:{}".format(epoch,epochs,float(val_acc)))
        if float(val_acc) > best_metric:
            best_metric = float(val_acc)
            model.save('path_to_location.h5')
            val_acc_metric.reset_states()



[0/50],val_acc:0.0
[0/50],val_acc:0.5
[0/50],val_acc:0.6666666865348816
[0/50],val_acc:0.75
[0/50],val_acc:0.6000000238418579
[0/50],val_acc:0.6666666865348816
[0/50],val_acc:0.5714285969734192
[0/50],val_acc:0.625
[0/50],val_acc:0.6666666865348816
[0/50],val_acc:0.699999988079071
[0/50],val_acc:0.7272727489471436
[0/50],val_acc:0.75
[0/50],val_acc:0.7692307829856873
[0/50],val_acc:0.7857142686843872
[0/50],val_acc:0.800000011920929
[0/50],val_acc:0.8125
[0/50],val_acc:0.8235294222831726
[0/50],val_acc:0.8333333134651184
[0/50],val_acc:0.7894737124443054
[0/50],val_acc:0.800000011920929
[0/50],val_acc:0.8095238208770752
[0/50],val_acc:0.7727272510528564
[0/50],val_acc:0.739130437374115
[0/50],val_acc:0.7083333134651184
[0/50],val_acc:0.7200000286102295
[0/50],val_acc:0.7307692170143127
[0/50],val_acc:0.7037037014961243
[0/50],val_acc:0.7142857313156128
[0/50],val_acc:0.7241379022598267
[0/50],val_acc:0.7333333492279053
[0/50],val_acc:0.7096773982048035
[0/50],val_acc:0.71875
[0/50],val

In [ ]:
p

In [59]:
from keras.layers import Dense,LSTM,Conv2D,MaxPool2D,Input,Add,ReLU,Reshape,Flatten,Concatenate,concatenate
from keras.models import Sequential, Model
import keras
import numpy as np

In [10]:
x= keras.backend.constant(np.random.randn(1,100,128))
lstm_layer1= LSTM(512,return_sequences=True)
lstm_layer2=LSTM(512,return_sequences=False)
y1= lstm_layer1(x)
y2= lstm_layer2(x)
for ii in [x,y1,y2]:
    print(ii.shape)


(1, 100, 128)
(1, 100, 512)
(1, 512)


In [14]:
x=keras.backend.constant(np.random.randn(16,28,28,1))
n_filter=16
filter_size=2
cnn_layer1=Conv2D(n_filter,filter_size,input_shape=(28,28,1))
y1=cnn_layer1(x)
n_filter=32
filter_size=4
cnn_layer2=Conv2D(n_filter,filter_size,input_shape=(28,28,1))
y2=cnn_layer2(x)
for ii in [x,y1,y2]:
    print(ii.shape)

(16, 28, 28, 1)
(16, 27, 27, 16)
(16, 25, 25, 32)


In [18]:
x=keras.backend.constant(np.random.randn(16,28,28,1))
n_filter=32
filter_size=4
cnn_layer2=Conv2D(n_filter,filter_size,input_shape=(28,28,1))
y1=cnn_layer2(x)
pooling_layer=MaxPool2D(2)
y2=pooling_layer(y1)
for ii in [x,y1,y2]:
    print(ii.shape)


(16, 28, 28, 1)
(16, 25, 25, 32)
(16, 12, 12, 32)


In [31]:
x=keras.backend.constant(np.random.randn(256,256))
layer = [
    Input(shape=(256,256,1)),
    Reshape((64,64,-1)),
    Conv2D(32,5),
    MaxPool2D(2),
    Conv2D(1,1),
    Flatten(),
    Dense(16, activation='sigmoid')
]
model = Sequential(layer)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 64, 64, 16)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 60, 60, 32)        12832     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 30, 30, 1)         33        
                                                                 
 flatten_3 (Flatten)         (None, 900)               0         
                                                                 
 dense_3 (Dense)             (None, 16)                14416     
                                                      

In [49]:
layer1 = [
    Input(shape=(256,256,1)),
    Reshape((64,64,-1)),
    Conv2D(8,5),
]

layer2 = [
    Input(shape=(256,256,1)),
    Conv2D(filters=8,kernel_size=17),
    MaxPool2D(pool_size=4)
]


model1 = Sequential(layer1)
model2 = Sequential(layer2)
print(model1.summary())
print(model2.summary())
merged = Concatenate()([model1.output, model2.output])
merged.shape

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_20 (Reshape)        (None, 64, 64, 16)        0         
                                                                 
 conv2d_44 (Conv2D)          (None, 60, 60, 8)         3208      
                                                                 
Total params: 3,208
Trainable params: 3,208
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_45 (Conv2D)          (None, 240, 240, 8)       2320      
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 60, 60, 8)        0         
 g2D)                                                            
                       

TensorShape([None, 60, 60, 16])

In [60]:
def new_model():
    i = Input(shape=(256,256,1))
    r = Reshape((64,64,-1))
    c1 = Conv2D(8,5)
    c2 = Conv2D(8,17)
    m2 = MaxPool2D(pool_size=4)
    rr = r(i)
    c1o = c1(rr)
    c2o = c2(i)
    mxo = m2(c2o)
    out = concatenate([c1o, mxo], axis=-1)
    
    model = Model(inputs=i, outputs=out)
    return model
new_model().summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_44 (InputLayer)          [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 reshape_27 (Reshape)           (None, 64, 64, 16)   0           ['input_44[0][0]']               
                                                                                                  
 conv2d_59 (Conv2D)             (None, 240, 240, 8)  2320        ['input_44[0][0]']               
                                                                                                  
 conv2d_58 (Conv2D)             (None, 60, 60, 8)    3208        ['reshape_27[0][0]']         

In [68]:
import keras 
from keras.losses import MeanAbsoluteError, MeanSquaredError
mse=MeanSquaredError()
y_true = [[0,0.3],[0,1]]
y_pred = [[1,1],[1,0]]
z = mse(y_true, y_pred)
print(z)
print(z.numpy())

tf.Tensor(1, shape=(), dtype=int32)
1


In [69]:
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

convert("train-images.idx3-ubyte", "train-labels.idx1-ubyte","mnist_train.csv", 60000)
convert("t10k-images.idx3-ubyte", "t10k-labels.idx1-ubyte","mnist_test.csv", 10000)
